In [191]:
##!conda install -c conda-forge geopy --yes 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/Rattanathon/opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-22.11.1              |   py39h6e9494a_1         874 KB  conda-forge
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.3.0                |     pyhd8ed1ab_0          68 KB  conda-forge
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    ruamel.yaml-0.17.21        |   py39ha30fb19_2         173 KB  conda-forge
    ruamel.yaml.clib-0.2.7     |   py39ha30fb19_1         118 KB  conda-forge
    --------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import random
from geopy.geocoders import Nominatim

import sklearn.neighbors
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

#!conda install -c conda-forge folium=0.5.0 --yes

import folium
import warnings
warnings.filterwarnings("ignore")

In [2]:
evcs = pd.read_csv ('./Greater_Melbourne.csv')
evcs.head(2)

,name,address,postcode,latitude,longitude,networks,total_plugs,Plugs_CCS_SAE,Plugs_CHAdeMO,Plugs_Caravan_Mains_Socket,Plugs_J1772,Plugs_Tesla,Plugs_Three_Phase,Plugs_Commando,Plugs_Type2,Plugs_Wall_AU_NZ,Plugs_Other,power_outputs_kw
0,Mornington Supercharger,"75 Mornington-Tyabb Rd, Mornington VIC 3931, A...",3931,-38.234305,145.051085,['Supercharger'],6,0,0,0,0,6,0,NaN,0,0,NaN,[250.0]
1,Sorrento Community Centre,"860 Melbourne Rd, Sorrento VIC 3943, Australia",3943,-38.338560,144.737043,['Evie Networks'],8,4,4,0,0,0,0,NaN,0,0,NaN,[50.065]


In [3]:
evcs_loc = evcs[['name','latitude', 'longitude']].copy()
evcs_loc.head(2)

,name,latitude,longitude
0,Mornington Supercharger,-38.234305,145.051085
1,Sorrento Community Centre,-38.338560,144.737043


In [4]:
shop_loc = pd.read_csv ('./shop_mel_drop.csv')
shop_loc.head(2)

,name,latitude,longitude,address
0,Emporium Melbourne,-37.812214,144.963785,"287 Lonsdale St, Melbourne VIC 3000"
1,Royal Arcade,-37.814368,144.964103,"335 Bourke St, Melbourne VIC 3000"


In [5]:
sport_loc = pd.read_csv ('./sport_mel_clean.csv')
sport_loc.head(2)

,name,latitude,longitude,address
0,Melbourne Cricket Ground (MCG),-37.819876,144.983414,"Brunton Ave, Melbourne VIC 3002"
1,Fitness First Flinders Street,-37.817785,144.965099,"276 Flinders St, Melbourne VIC 3000"


In [6]:
travel_loc = pd.read_csv ('./travel_mel_clean.csv')
travel_loc.head(2)

,name,latitude,longitude,address
0,Melbourne Airport (MEL),-37.669714,144.848484,"Departure Dr (Tullamarine Fwy), Tullamarine VI..."
1,Crown Metropol,-37.825752,144.957646,"8 Whiteman St, Southbank VIC 3006"


In [7]:
places_loc = pd.concat([shop_loc, sport_loc,travel_loc], ignore_index=True)
places_loc.rename(columns={'name':'POI Name'}, 
                 inplace=True)
places_loc.head(2)

,POI Name,latitude,longitude,address
0,Emporium Melbourne,-37.812214,144.963785,"287 Lonsdale St, Melbourne VIC 3000"
1,Royal Arcade,-37.814368,144.964103,"335 Bourke St, Melbourne VIC 3000"


In [8]:
places_loc.drop(places_loc.columns[3], axis=1, inplace=True)
places_loc.shape

(293, 3)

In [9]:
#convert coordinates to distance in radius
evcs_loc[['latitude', 'longitude']] = (
    np.radians(evcs_loc.loc[:,['latitude', 'longitude']])
)
places_loc[['latitude', 'longitude']] = (
    np.radians(places_loc.loc[:,['latitude', 'longitude']])
)
evcs_loc.head(2)

,name,latitude,longitude
0,Mornington Supercharger,-0.667315,2.531619
1,Sorrento Community Centre,-0.669134,2.526138


In [10]:
places_loc.head(2)

,POI Name,latitude,longitude
0,Emporium Melbourne,-0.659948,2.530095
1,Royal Arcade,-0.659985,2.530101


In [11]:
#create distance metric between each pairs of ev charging stations and venues using Haversine distance.
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')
dist_matrix = (dist.pairwise
    (evcs_loc[['latitude', 'longitude']],
     places_loc[['latitude', 'longitude']])*6371
)
dist_matrix_df = (
    pd.DataFrame(dist_matrix,index=evcs_loc['name'], 
                 columns=places_loc['POI Name'])
)
dist_matrix_df.head(5)

POI Name,Emporium Melbourne,Royal Arcade,The Clarendon Centre,Melbourne Central,Homeplus+ Homemaker Centre,Barkly Square Shopping Centre,Warringal Shopping Centre,M-City,Qv,Moonee Ponds Central,...,Manna Heights Bed & Breakfast,Meadowood B&B,Orchard Keepers,Capel Sound Escapes - Westend Apartment,Air Communications,Choice Hotels Australasia,Buoys on Queens,Kinvara B&B,Boutique Stays - Brighton Retreat,Espresso Apartments
name,,,,,,,,,,,,,,,,,,,,,
Mornington Supercharger,47.553258,47.312365,45.384951,47.761660,39.844295,51.607216,53.231710,35.736805,47.684503,53.272448,...,14.423911,16.983143,17.635427,21.002239,28.990408,31.631210,30.246349,34.476539,36.034684,36.605838
Sorrento Community Centre,61.800636,61.582742,59.592679,61.966531,59.972051,65.587123,71.013448,58.339636,61.996116,65.711553,...,20.614688,25.083486,27.506889,12.456956,50.956401,57.408905,0.692553,10.292746,52.064054,53.216425
CTR Whittlesea,36.195734,36.407253,38.377785,36.038581,40.858593,32.575844,27.612833,45.642740,35.995350,33.213560,...,94.057032,97.611524,98.312234,97.247050,51.763849,52.472477,98.646829,93.510586,46.302345,45.317089
Rye & Sorrento Camping,60.483954,60.269966,58.295243,60.644947,59.176203,64.215668,69.909846,57.806730,60.682541,64.223743,...,22.447379,27.151717,29.592623,14.596066,50.450021,57.253634,2.982885,7.711771,51.007301,52.191284
McDonalds Melton South,35.823074,35.916609,36.257272,35.699054,48.849962,34.670113,43.811852,54.538037,35.921896,31.197230,...,77.944516,83.351437,85.222912,76.105935,53.673813,62.444636,70.554206,61.239375,42.271276,42.912393


In [12]:
#Pick only the pairs with distance less than 5 km.
indicator = dist_matrix_df.le(5).astype(int)
indicator.shape

(237, 293)

In [13]:
#Find total number of nearby(distance less than 5 km) evcs for each venues.
sum_evcs = indicator.sum(axis = 0)
sum_evcs.head(242)

POI Name
Emporium Melbourne            50
Royal Arcade                  50
The Clarendon Centre          51
Melbourne Central             50
Homeplus+ Homemaker Centre     9
                              ..
Ferntree Gully Hotel           1
Old England Hotel              5
Greensborough Motor Inn        8
Quest Dandenong Central        7
Tower Hotel                   11
Length: 242, dtype: int64

In [14]:
#Get venues where there are less than or 2 evcs stations nearby.
places_names = sum_evcs[(sum_evcs <= 2)]
places_names

POI Name
Westfield Knox                           2
Roxburgh Village                         1
Craigieburn Central                      2
Stockland Highlands                      2
Westfield                                2
Riverdale Village Shopping Centre        1
Watergardens                             2
Derrimut Village Shopping Centre         2
Greenvale Shopping Centre                2
Taylors Hill Village                     2
Home Consortium Braybrook                1
Braybrook Shopping Centre                2
Mountain Gate Shopping Centre            1
Wellington Village Shopping Centre       2
Merrindale Shopping Centre               1
Dentist Boronia                          1
Studfield Shopping Strip                 2
Healesville Walk Shopping Centre         0
Langwarrin Plaza                         0
The Avenue Village Shopping Centre       2
Carrum Downs Regional Shopping Centre    0
Centro Somerville                        0
Somerville Plaza Travel                  0
Th

In [15]:
#Find corresponindg coordinators of the selected venues
places_selected = places_loc.loc[places_loc['POI Name'].isin(places_names.index)]
#convert the latitudes and longitudes back from radius to degrees
places_selected[['latitude', 'longitude']] = (np.degrees(places_loc.loc[:,['latitude', 'longitude']]))
places_selected.head(2)

,POI Name,latitude,longitude
20,Westfield Knox,-37.868995,145.240171
27,Roxburgh Village,-37.639439,144.932380


In [16]:
places_selected.shape

(60, 3)

In [17]:
#Create clusters for selected venues

number_of_clusters = 10
places_selected_xys = places_selected[['latitude', 'longitude']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(places_selected_xys)
clusters = kmeans.cluster_centers_

print(clusters)
print(kmeans.labels_)

[[-38.111692   145.21265218]
 [-37.64005329 144.92155871]
 [-37.65665475 145.4477535 ]
 [-38.41053433 145.05345533]
 [-38.338178   144.7507285 ]
 [-37.8735712  145.2849056 ]
 [-38.02724743 145.42703371]
 [-38.4103435  145.3582325 ]
 [-37.910748   144.9946865 ]
 [-37.764756   144.77579917]]
[5 1 1 1 1 9 9 1 9 1 9 9 9 5 5 5 5 5 2 0 0 0 0 0 0 0 0 0 0 6 5 6 6 6 6 0 0
 0 0 1 5 6 5 2 2 5 2 6 4 3 0 0 3 7 3 7 0 4 8 8]


In [18]:
new_evcs = places_selected
new_evcs.insert(0, 'Cluster Labels', kmeans.labels_)

In [19]:
new_evcs.head(2)

,Cluster Labels,POI Name,latitude,longitude
20,5,Westfield Knox,-37.868995,145.240171
27,1,Roxburgh Village,-37.639439,144.932380


In [20]:
# clusters map

evcs_clusters = folium.Map(location=[-37.8136, 144.9631], zoom_start=8)

# set clusters colors
x = np.arange(number_of_clusters)
ys = [i + x + (i*x)**2 for i in range(number_of_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lon, poi, cluster in zip(new_evcs['latitude'], new_evcs['longitude'], new_evcs['POI Name'], new_evcs['Cluster Labels']):
    name = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=name,
        tooltip = name,
        color=rainbow[cluster-1],
        fill=True,
        fill_opacity=0.8).add_to(evcs_clusters)
       
evcs_clusters

In [24]:
geolocator = Nominatim(user_agent="foursquare_agent")

lat = 35.74137968
long = -78.59448015

evcs_new = []

print("Suggsted new locations for EV charging stations are: \n")
for lat, long in clusters:
    location = geolocator.reverse([lat, long])
    evcs_new.append(
        {
            'Address': location,
            'Latitude': lat,
            'Longitude':  long
        }
     )
    print(location.address, "\n")
    
evcs_new = pd.DataFrame(evcs_new)

Suggsted new locations for EV charging stations are: 

Taylors Road, Skye, Melbourne, City of Frankston, Victoria, 3910, Australia 

Broadmeadows Valley Trail, Meadow Heights, Melbourne, City of Hume, Victoria, 3048, Australia 

Healesville - Yarra Glen Road, Tarrawarra, Melbourne, Shire of Yarra Ranges, Victoria, 3775, Australia 

Frankston - Flinders Road, Shoreham, Melbourne, Shire of Mornington Peninsula, Victoria, 3916, Australia 

Point Nepean Road, Sorrento, Melbourne, Shire of Mornington Peninsula, Victoria, 3943, Australia 

Dorset Road, Boronia, Melbourne, City of Knox, Victoria, 3156, Australia 

Turners Lane, Officer, Melbourne, Shire of Cardinia, Victoria, 3808, Australia 

French Island, Victoria, 3921, Australia 

Outer Crescent, Brighton, Melbourne, City of Bayside, Victoria, 3186, Australia 

Station Road, Cairnlea, Melbourne, City of Brimbank, Victoria, 3023, Australia 



In [238]:
evcs_new.to_csv('new_location.csv',index=False)

In [36]:
# new evcs map

evcs_map1 = folium.Map(location=[-37.8136, 144.9631], zoom_start=8)

for _, evcs in evcs_new.iterrows():
    folium.Marker(location = [evcs['Latitude'], evcs['Longitude']],
                 popup = evcs['Address']).add_to(evcs_map1)

evcs_map1